# 1 Exploratory Data Analysis - Bank Clients Credibility Assessment 

The customer is the credit department of the bank. The goal of the study it to find out whether the marital status, the purpose of the loan, the level of income and the number of children affect the fact of repaying the loan on time.

## Contents
<ol>
<li>Describing the Data</li>
<li>Data Preprocessing</li>
<li>Exploratory Data Analysis</li>
<li>Conclusion</li>
</ol>

## 1. Describing the Data

In [1]:
import pandas as pd

In [2]:
logs=pd.read_csv('loans.csv')
logs.head()

,children,dob_years,education,marital_status,gender,income_type,debt,total_income,purpose
0,1,42,GRADUATE,married,F,employee,0,253875.6395,buying a home
1,1,36,high school,married,F,employee,0,112080.0141,car purchase
2,0,33,high school,married,M,employee,0,145885.9523,buying a home
3,3,32,high school,married,M,employee,0,267628.5503,degree
4,0,53,high school,married,F,retiree,0,158616.0779,get married


In [3]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   children        21525 non-null  int64  
 1   dob_years       21525 non-null  int64  
 2   education       21525 non-null  object 
 3   marital_status  21525 non-null  object 
 4   gender          21525 non-null  object 
 5   income_type     21525 non-null  object 
 6   debt            21525 non-null  int64  
 7   total_income    19351 non-null  float64
 8   purpose         21525 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1.5+ MB


The dataset has 8 variables and 21525 records. Variables include age, marital status, the amount of children, level of education, gender, income, type of income and employment, the purpose of the requested loan and status of the loan.

In [4]:
logs.describe()

,children,dob_years,debt,total_income
count,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,43.293380,0.080883,1.674223e+05
std,1.381587,12.574584,0.272661,1.029716e+05
min,-1.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,33.000000,0.000000,1.030532e+05
50%,0.000000,42.000000,0.000000,1.450179e+05
75%,1.000000,53.000000,0.000000,2.034351e+05
max,20.000000,75.000000,1.000000,2.265604e+06


**dob_years** - the age in years, int variable, has null values

**total_income** - numerical variable, should be converted into int and grouped by the levels of income, has outliers and gaps

In [5]:
logs['debt'].value_counts().sort_values()

1     1741
0    19784
Name: debt, dtype: int64

**debt** - categorical variable, has no gaps

19784 of clients repaid the debts on time, 1741 - no

In [6]:
logs['children'].value_counts().sort_values()

 5         9
 4        41
-1        47
 20       76
 3       330
 2      2055
 1      4818
 0     14149
Name: children, dtype: int64

**children** - the amount of children the client has, numeric value. 47 clients have -1 (that should be a mistake) and 76 clients have 20 (which also looks like a mistake)

In [7]:
logs['education'].value_counts().sort_values()

High School             3
phd                     6
secondary school      282
underGRADUATE         744
GRADUATE             5260
high school         15230
Name: education, dtype: int64

**education** - categorical variable (phd, secondary school, undergraduate, graduate, high school), no gaps, values should be lowercased

In [8]:
logs['marital_status'].value_counts().sort_values()

widow / widower      960
divorced            1195
single              2813
married            16557
Name: marital_status, dtype: int64

**marital_status** - categorical variable (widow/widower, divorced, signle or married), no gaps

In [9]:
logs['gender'].value_counts().sort_values()

XNA        1
M       7288
F      14236
Name: gender, dtype: int64

**gender** - categorical variable, has 2 values (M and F), 1 outlier, no gaps

In [10]:
logs['income_type'].value_counts().sort_values()

public employee     1459
retiree             3856
unemployed          5089
employee           11121
Name: income_type, dtype: int64

**income_type** - categorical variable (public employee, retiree, unemployed, employee), no gaps

In [11]:
logs['debt'].value_counts().sort_values()

1     1741
0    19784
Name: debt, dtype: int64

**debt** - categorical variable. 0 if the loan is repaid (19784 clients), 1 if not (1741 clients), no gaps

In [12]:
logs['purpose'].value_counts().sort_values()

specialized education                       436
car deal                                    455
degree                                      462
car purchase                                462
cars                                        478
to buy a used car                           479
car                                         480
used car deal                               489
automobile                                  495
purchase of residential property            607
home renovation                             612
buying home                                 620
real estate                                 620
buying a property                           624
construction of residential real estate     626
transactions with real estate               630
property                                    634
house construction                          635
buying a family home                        641
buying a home                               647
housing                                 

**purpose** - the purpose of the loan, categorical variable, has duplicates

## Step 2. Data Preprocessing

### 2.1 Replacing 'XNA' with "F" as this dataset has almost twice as many women as men

In [13]:
logs.loc[logs['gender'] == 'XNA', 'gender'] = "F"
logs['gender'].value_counts().sort_values()

M     7288
F    14237
Name: gender, dtype: int64

### 2.2 Replacing value -1 with 0 in children

Let's consider these negative values as the absence of children

In [14]:
logs.loc[logs['children'] == -1, 'children'] = 0
logs['children'].value_counts().sort_values()

5         9
4        41
20       76
3       330
2      2055
1      4818
0     14196
Name: children, dtype: int64

### 2.3 Handling nulls in dob_years

The variable dob_years (age) contains 101 zero values, let's check if these gaps are random

In [15]:
logs.loc[logs['dob_years'] == 0]['income_type'].value_counts()

employee           55
unemployed         20
retiree            20
public employee     6
Name: income_type, dtype: int64

Gap values occur in all categories of employment, let's replace them with the average values per employment type, rounded to the nearest integer

In [16]:
employee=logs.loc[logs['income_type'] == 'employee']['dob_years'].mean()
unemployed=logs.loc[logs['income_type'] == 'unemployed']['dob_years'].mean()
retiree=logs.loc[logs['income_type'] == 'retiree']['dob_years'].mean()
public_employee=logs.loc[logs['income_type'] == 'public employee']['dob_years'].mean()

logs.loc[(logs['dob_years'] == 0) & (logs['income_type'] == 'employee'), 'dob_years'] = round(employee)
logs.loc[(logs['dob_years'] == 0) & (logs['income_type'] == 'unemployed'), 'dob_years'] = round(unemployed)
logs.loc[(logs['dob_years'] == 0) & (logs['income_type'] == 'retiree'), 'dob_years'] = round(retiree)
logs.loc[(logs['dob_years'] == 0) & (logs['income_type'] == 'public employee'), 'dob_years'] = round(public_employee)

### 2.4 Handling nulls in total_income

total_income vatiable contains 2174 gaps, let's check if these gaps are random

In [17]:
logs['total_income'].fillna(0, inplace=True)
logs.loc[logs['total_income'] == 0]['income_type'].value_counts()

employee           1106
unemployed          508
retiree             413
public employee     147
Name: income_type, dtype: int64

Gap values occur in all categories of employment, let's replace them with the average values per employment type

In [18]:
logs.loc[(logs['income_type'] == 0) & (logs['income_type'] == 'employee'), 'total_income'] = logs.loc[logs['income_type'] == 'employee']['total_income'].mean()
logs.loc[(logs['income_type'] == 0) & (logs['income_type'] == 'unemployed'), 'total_income'] = logs.loc[logs['income_type'] == 'unemployed']['total_income'].mean()
logs.loc[(logs['income_type'] == 0) & (logs['income_type'] == 'retiree'), 'total_income'] = logs.loc[logs['income_type'] == 'retiree']['total_income'].mean()
logs.loc[(logs['income_type'] == 0) & (logs['income_type'] == 'public employee'), 'total_income'] = logs.loc[logs['income_type'] == 'public employee']['total_income'].mean()

In [19]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   children        21525 non-null  int64  
 1   dob_years       21525 non-null  int64  
 2   education       21525 non-null  object 
 3   marital_status  21525 non-null  object 
 4   gender          21525 non-null  object 
 5   income_type     21525 non-null  object 
 6   debt            21525 non-null  int64  
 7   total_income    21525 non-null  float64
 8   purpose         21525 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1.5+ MB


###  2.5 Changing Data Types 

Let's convert total_income into integer

In [20]:
logs['total_income']=round(logs['total_income']).astype('int64')

### 2.6 Lowercasing the values of education

In [21]:
logs['education']=logs['education'].str.lower()
logs['education'].value_counts().sort_values()

phd                     6
secondary school      282
undergraduate         744
graduate             5260
high school         15233
Name: education, dtype: int64

### 2.7 Data Classification - Children

Let's create additional categorial variable - has_children (**0** - for no or **1** - for yes)

In [22]:
logs.loc[logs['children'] == 0, 'has_children'] = 0
logs.loc[logs['children'] != 0, 'has_children'] = 1
logs['has_children']=round(logs['has_children']).astype('int64')

logs['has_children'].value_counts().sort_values()

1     7329
0    14196
Name: has_children, dtype: int64

### 2.8 Data Classification - Income

Let's create additional categorial variable - income_level. We'll check the minimum, max, median and quantile values of the income variable to determine the levels

In [23]:
print(logs['total_income'].min())
print(logs['total_income'].quantile(.2))
print(logs['total_income'].median())
print(logs['total_income'].quantile(.8))
print(logs['total_income'].max())

income_max = logs['total_income'].max()

0
78561.0
135515.0
214254.2
2265604


80% of all values lie in the range from 0 to 250,0000, to get 6 steps of income, we will break customers into steps of 50K

In [29]:
logs.loc[logs['total_income'] == 0, 'income_level'] = 0
logs.loc[(logs['total_income'] > 0) & (logs['total_income'] < 50000), 'income_level'] = '<50K'
logs.loc[(logs['total_income'] >= 50000) & (logs['total_income'] <= 100000), 'income_level'] = '50K-99,99K'
logs.loc[(logs['total_income'] >= 100000) & (logs['total_income'] <= 150000), 'income_level'] = '100K-149,99K'
logs.loc[(logs['total_income'] >= 150000) & (logs['total_income'] <= 200000), 'income_level'] = '150K-199,99K'
logs.loc[(logs['total_income'] >= 200000) & (logs['total_income'] <= 250000), 'income_level'] = '200K-249,99K'
logs.loc[(logs['total_income'] >= 250000) & (logs['total_income'] <= income_max), 'income_level'] = '>=250K'

print (logs['income_level'].value_counts().sort_values())

<50K             372
0               2174
200K-249,99K    2254
>=250K          2812
50K-99,99K      4091
150K-199,99K    4118
100K-149,99K    5704
Name: income_level, dtype: int64


### 2.9 Data Classification - Purpose

Let's classify all purposes by keywords

In [33]:
logs.loc[logs['purpose'].str.contains('car'), 'purpose'] = 'car'
logs.loc[logs['purpose'].str.contains('automobile'), 'purpose'] = 'car'
logs.loc[logs['purpose'].str.contains('home'), 'purpose'] = 'real estate'
logs.loc[logs['purpose'].str.contains('house'), 'purpose'] = 'real estate'
logs.loc[logs['purpose'].str.contains('estate'), 'purpose'] = 'real estate'
logs.loc[logs['purpose'].str.contains('housing'), 'purpose'] = 'real estate'
logs.loc[logs['purpose'].str.contains('property'), 'purpose'] = 'real estate'
logs.loc[logs['purpose'].str.contains('education'), 'purpose'] = 'education'
logs.loc[logs['purpose'].str.contains('degree'), 'purpose'] = 'education'
logs.loc[logs['purpose'].str.contains('wedding'), 'purpose'] = 'wedding'
logs.loc[logs['purpose'].str.contains('married'), 'purpose'] = 'wedding'
logs['purpose'].value_counts().sort_values()

wedding         2348
education       4022
car             4315
real estate    10840
Name: purpose, dtype: int64

## 3. Exploratory Data Analysis

### 3.1 Is there a dependency between having children and repaying the loan on time?

In [34]:
table_children = logs.groupby('has_children')['debt'].mean()*100
table_children

has_children
0    7.495069
1    9.237277
Name: debt, dtype: float64

**Conclusion:** The clients without children repay the debts on time more often than the clients with children (9.23% of non-payers against 7.5%).

### 3.2 Is there a dependency between marital status and repaying the loan on time?

In [36]:
table_family_status = logs.groupby('marital_status')['debt'].mean()*100
table_family_status.sort_values(ascending=False)

marital_status
single             9.740491
married            7.966419
divorced           7.112971
widow / widower    6.562500
Name: debt, dtype: float64

**Conclusion:** Widows and widowers repay the debts on time more often than the married ones, divorced ones and singles (9.74%, 7.97%, 7.11% and 6.56% respectively)

### 3.3 Is there a dependency between the level of income and repaying the loan on time?

In [37]:
table_income_level = logs.groupby('income_level')['debt'].mean()*100
table_income_level.sort_values(ascending=False)

income_level
150K-199,99K    8.936377
100K-149,99K    8.607994
50K-99,99K      8.090931
0               7.819687
200K-249,99K    7.275954
>=250K          6.899004
<50K            6.182796
Name: debt, dtype: float64

**Conclusion** The higher the income, the lower the percentage of returned loans among clients with income from 1 to 200,000

The percentage of non-payers among clients with income > 200,000 is higher than among clients with income from 50,001 to 200,000, but lower than for clients with income from 0 to 50,000

### 3.4 Is there a dependency between the purpose of the loan and repaying the loan on time?

In [38]:
table_income_level = logs.groupby('purpose')['debt'].mean()*100
table_income_level.sort_values(ascending=False)

purpose
car            9.339513
education      9.199403
wedding        7.921635
real estate    7.214022
Name: debt, dtype: float64

The student loan repayment rate is roughly equal to the car loan repayment rate (9.34% against 9.2% of non-payers). 

The wedding  loan repayment rate is roughly equal to the real estate loan repayment rate (7.92% against 7.21% of non-payers). 

## Overall Conclusion

More than 90% of all clients repay loans on time. Children, marital status, income level and the purpose of the loan have little effect on the repayment of loans on time, increasing or decreasing the chance of non-payments by only 1-2%.

To simplify further work with the data, I would advise the bank to add picklists and conditions for the following variables: purpose (car, real estate, education, wedding), education (all values in lowercase letters), gender (only F or M), and rename the dob_years variable to age.